In [ ]:
from huggingface_hub import login
login(token="")

c:\Users\USER\anaconda3\envs\vectordb_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import torch
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)

In [2]:
# gpu 사용 설정을 위한 device 변수 생성
# 쿠다가 사용 가능한 환경인지 확인하고 가능하면 쿠다 아니면 cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 병렬 토큰나이저 경고 방지
os.environ["TOKENIZERS_PARALLELISM"] = "false"

cpu


In [14]:
# 모델 로드
model_name = "CarrotAI/Llama-3.2-Rabbit-Ko-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForSequenceClassification.from_pretrained(model_name)
base_model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at CarrotAI/Llama-3.2-Rabbit-Ko-3B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# NF4 양자화를 위한 설정
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True, # 모델을 4비트 정밀도로 로드
    bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
    bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
    bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=nf4_config
).to(device)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [10]:
import json
import pandas as pd
from datasets import Dataset
with open ("finetunning_tmp.json", "r", encoding='UTF-8-sig') as f:
    data = json.load(f)
data[:5]

[{'question': '하나와 롯데 자동차 보험의 어린이 보호구역 사고 시 과실 인정 기준 차이는?',
  'answer': '하나손해보험과 롯데손해보험의 약관에서 어린이 보호구역 사고 시 과실 인정 기준에 대한 구체적인 정보는 주어지지 않았습니다. \n\n하지만 일반적인 자동차 보험의 경우, 어린이 보호구역 내에서 사고가 발생하면 운전자에게 보다 엄격한 과실 기준이 적용될 수 있으며, 이는 보험사마다 다르게 적용될 수 있습니다. 일반적으로 어린이 보호구역에서는 운전자가 주의 의무를 더욱 철저히 해야 하며, 사고 발생 시 운전자에게 더 큰 책임이 부과될 가능성이 높습니다. \n\n정확한 정보를 얻기 위해서는 각 보험사에 직접 문의하거나, 해당 보험사의 자동차 보험 약관을 자세히 살펴보시는 것이 좋습니다.'},
 {'question': '하나손해보험과 롯데손해보험의 교통사고 처리 지원금 보장 횟수와 대상 범위 차이는?',
  'answer': '현재 제공된 context 정보에서는 하나손해보험과 롯데손해보험의 교통사고 처리 지원금에 대한 보장 횟수와 대상 범위에 대한 구체적인 차이를 확인할 수 없습니다. \n\nHANA_context와 LOTTE_context에 제공된 정보는 주로 보험 상품에 대한 일반적인 안내, 예금자 보호에 관한 정보, 사고 접수 및 보험 처리 관련 안내 등이 포함되어 있으며, 교통사고 처리 지원금에 대한 구체적인 보장 횟수나 대상 범위에 대한 정보는 명시되어 있지 않습니다.\n\n보험 약관의 세부 내용은 각 보험사의 공식 웹사이트나 고객센터를 통해 확인하실 수 있으며, 필요시 직접 보험사에 문의하시는 것이 정확한 정보를 얻는 방법입니다. 하나손해보험의 고객센터 전화번호는 1566-3000, 롯데손해보험의 고객센터 전화번호는 1588-3344 또는 1600-3434입니다.'},
 {'question': '하나손해보험과 롯데손해보험의 예방접종 후 부작용 보장 여부 비교해줘.',
  'answer': '하나손해보험과 롯데손해보험의 약관 내용

In [11]:
df = pd.DataFrame(data)

# HuggingFace Dataset 형태로 변환
df_train = Dataset.from_pandas(df)
df_train

Dataset({
    features: ['question', 'answer'],
    num_rows: 16
})

In [ ]:
def preprocess_data(texts, labels):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    encodings["lables"] = torch.tensor(labels, dtype=torch.long)
    return encodings